In [8]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches as mpatches
from sklearn import preprocessing
import wfdb
import copy as cp
import scipy.signal as signal
from scipy import stats
from sklearn import preprocessing
from tqdm import tqdm
import os
import re
import pandas as pd
import pickle
import csv

In [9]:
rlist = []
records = os.path.normpath('mit-bih-dataframes/subject_list.csv')
with open(records) as rfile:
    recordreader = csv.reader(rfile, delimiter=' ', quotechar='|')
    for row in recordreader:
        rlist.append(row[0])

In [10]:
subject_dfs = {}
for record in tqdm(rlist):
    subject_dfs[record] = pd.read_csv(os.path.normpath('mit-bih-extracted/'+record+'_full.csv'), index_col=[0,1])

  0%|                                                    | 0/23 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'mit-bih-extracted/04015_full.csv'

In [ ]:
def plot_time_plot(data, selection, column, nrows, ncols, title, xlabel, ylabel):
    plt.figure(figsize=(28, 16), constrained_layout=True)
    
    for idx, record in enumerate(selection):
        ax = plt.subplot(nrows, ncols, idx + 1)
        ax.set_title('Subject '+str(record), fontsize=28)
        ax.set_xlabel(xlabel, fontsize=24)
        ax.set_ylabel(ylabel, fontsize=24)
        ax.tick_params(labelsize=22)

        df = data[record]

        current = df['rhythmLabel'].iloc[0]
        lastidx = 0
        print("Plot "+str(idx+1)+", Subject "+str(record))
        for row in tqdm(df.itertuples()):
            pos = row.Index[0]
            if pos == len(df)-1:
                dataset=df.loc[lastidx:pos+1]
                if current == 'N':
                    ax.plot(dataset.index.get_level_values('timestamp'), dataset[column], color='darkcyan')
                elif current == 'A':
                    ax.plot(dataset.index.get_level_values('timestamp'), dataset[column], color='orange')
                else:
                    ax.plot(dataset.index.get_level_values('timestamp'), dataset[column], color='salmon')
            elif row.rhythmLabel == current:
                continue
            else:
                dataset=df.loc[lastidx:pos]
                if current == 'N':
                    ax.plot(dataset.index.get_level_values('timestamp'), dataset[column], color='darkcyan')
                elif current == 'A':
                    ax.plot(dataset.index.get_level_values('timestamp'), dataset[column], color='orange')
                else:
                    ax.plot(dataset.index.get_level_values('timestamp'), dataset[column], color='salmon')
                lastidx = pos
                current = row.rhythmLabel
        ax.axhline(y=df[df['rhythmLabel'] == 'N'][column].mean(), color="darkslategray", linestyle="--", linewidth=5)
        ax.axhline(y=df[df['rhythmLabel'] == 'A'][column].mean(), color="purple", linestyle="-.", linewidth=5)
        ax.axhline(y=df[df['rhythmLabel'] == 'O'][column].mean(), color="darkred", linestyle="--", linewidth=5)

    plt.suptitle(title, fontsize=36)

    normal_patch = mpatches.Patch(color='darkcyan', label='Normal')
    afib_patch = mpatches.Patch(color='orange', label='Afib')
    other_patch = mpatches.Patch(color='salmon', label='Other')
    normal_mean_patch = mpatches.Patch(color='darkslategray', label='Normal mean')
    afib_mean_patch = mpatches.Patch(color='purple', label='Afib mean')
    other_mean_patch = mpatches.Patch(color='darkred', label='Other mean') 
    plt.figlegend(handles=[normal_patch, afib_patch, other_patch, normal_mean_patch, afib_mean_patch, other_mean_patch], 
                #['Normal', 'Afib', 'Other', 'Normal mean', 'Afib mean', 'Other mean'],
                fontsize=28, 
                loc = "upper right",
                ncol=2,
                shadow = True)

In [ ]:
plot_time_plot(subject_dfs, ['04015'], 'rr_int_seconds', 1, 1, 'Primarily Normal', 'Seconds', 'RR-Interval length (seconds)')
plot_time_plot(subject_dfs, ['07859'], 'rr_int_seconds', 1, 1, 'Primarily Afib', 'Seconds', 'RR-Interval length (seconds)')
plot_time_plot(subject_dfs, ['04043'], 'rr_int_seconds', 1, 1, 'Both Prevalent', 'Seconds', 'RR-Interval length (seconds)')
plot_time_plot(subject_dfs, ['04936'], 'rr_int_seconds', 1, 1, 'Irregular', 'Seconds', 'RR-Interval length (seconds)')